### C++的内存概念

![1](img/mem0001.png)
![1](img/mem0002.png)
![1](img/mem0003.png)
![1](img/mem01.png)
![1](img/mem02.png)
![1](img/mem1.png)
![1](img/mem2.png)
![1](img/mem3.png)
![1](img/mem4.png)

内存怎么表示负数？

- 正数正常表示，符号位设0
- 负数先用正数表示，符号位设1, 然后其余数字全部取反
- 最后再加1

10转二进制：
- 10/2=5余0
- 5/2=2余1
- 2/2=1余0
- 1/2=0余1
- 从后向前，得：1010
- 如果是8bit表示一个有符号整数，那么结果是：0000 1010

那么-10在内存里是什么样的？
- 从上面知道+10是: 0000 1010
- 设符号位为1: 1000 1010
- 取反其它位: 1111 0101
- 加1: 1111 0110

一个指针指向一个地址，如果地址是32位的，那么它最大可以表示多大？
($2^{32} -1$)，约等于4G  
所以在32位的时代，只能处理最多4G的内存。


### 变参

```c++
void print(const char* fmt, ...) {
    auto p = (unsigned char*)&fmt + sizeof(fmt)
    for(int i = 0; i < 32; ++i) {
        printf("%02X, ", p[i];
    }
}
```
这里，p先是取了第一个参数的位置，再加上参数的长度，就指向了参数结尾的位置
然后开始逐（字节？）打印内存里面的值
所以变参没有靠名字去取，而是从内存里取
这个内存地址就是栈空间地址

![image.png](img/mem5.png)

### 结构体

{}和=都能初始化结构体的属性，{}是c11语法，但它可以支持更加复杂的初始化场景（基于`std::initializer_list`)

```cpp
struct Animal{
    int age{18};
    int money = 100;
    string name{"hello"};
};
```

struct定义后之所以要接分号，是因为它的定义还存在如下情况：
```cpp
struct obj {} my_obj;
```
即定义时直接实例化给一个变量，所以如果你不加分号，编译器会认为后面的部分是一个变量，会把struct给它。

而普通函数的话，会在}结尾，就没这个二义性，可以不加分号。

### 字节对齐

```cpp
struct Animal {
        char __attribute__((aligned(4))) a = 0x0A;   // 字节对齐
        char __attribute__((aligned(4))) b = 0x0B;
    } animal;

cout << sizeof(animal) << endl;
```
如果没有字节对齐，animal就是2字节，对齐后就有8字节了。没对齐时，一次取N个字节就不能保证取到些什么，可能取a的时候就把b也取出来了

### 指针

```c++
int a  = 0xAABBCCDD;
int *pa = &a;
cout << pa << endl;             // 原生地址用16进制表示
cout << (long int)pa << endl;   // 强转成长整也可以
cout << sizeof(long int) <<  sizeof(int) << endl; // 看下长度
printf("%x\n", *pa);

```

- 先找到pa的值，即一个地址
- 再找到它定义时对应变量的类型（在哪里存的？肯定存了的）
- 发现是个int，于是就去取4个字节（32位）
- 取到的4个字节转换成int的类型

证明定义指针的时候是存了变量的：
```cpp
unsigned char * p = (unsigned char *)&a;
printf("%x\n", *p);
```
输出`dd`, 因为定义p的时候变量类型是char，所以只会取一个字节，因为小端的缘故，取的是最后一个字节，即dd
同样， *(p+1)表示右移`一个类型的长度`，这就是前面的for循环里++i能准确地向后找变量的原因，其实是定长移动。

即：
```cpp
p[2] == *(p + 2)
```

同理，也可以强转成别的形式，如`unsigned short *`，它是2字节一单位了  
也同理，正常定义指针的时候，其实是省略了`(int *)&animal`前面的强转的，因为它就是默认的

所以`unsigned short *`会一次取2字节，`int *`会一次取4字节（前面说过了）

以下证明索引加1，其实地址是加了2的（对于unsigned char)
```cpp
cout << spa << ", " << spa + 1 << endl; 
```
输出：0x7ffeed47ef`6c`, 0x7ffeed47ef`6e`，6c -> 6e，确实加的是2

### 指针的指针

- 地址是用8个字节表示的
- 所以指针的指针取的都是8个字节
- 取到最里层，就要看值的类型来取相应长度的字节了

```cpp
int** ppa = (int**)&p;
printf("%p\n", ppa);     // 这是个地址，取8个字节
printf("%p\n", *ppa);    // 这也是个地址
printf("0X%x\n", **ppa); // 这才是取值， 取4个字节
```


### new

![mem_new](img/mem_new.png)

- new 返回的是指针，所以才看到为什么调用类的属性要用->，见“基础”笔记本
- new 是向系统申请空间，是在堆上面。只有栈空间才是为代码块申请的空间，堆上的空间就是内存条上的独立空间（为程序申请的），所以程序运行完了也不会自动释放，程序结束才释放。

### malloc

```cpp
// malloc内存空间，至于要多少自己张口就是了 
// 申请的空间在堆上
// int* p2 = (int *)malloc(sizeof(int) * 3);
Animal* p = (Animal *)malloc(sizeof(Animal));
// init 语法奇葩，简单理解就是new的同时把申请的内存空间告诉系统
Animal* animal = new (p) Animal();

// p指针和animal指针指向的是同一个地址，但两个指针是不同的
// 因为指针本身也是个变量，这个变量也需要一个地址来存
cout << "p: "<< p <<  endl;                // 0x7fa2b1c05c80
cout << "animal: " << animal << endl;      // 0x7fa2b1c05c80
cout << "&p: " << &p << endl;              // 0x7ffee76c8f70 (different)
cout << "animal size: " << sizeof(*animal) << endl;

cout << "age: " << p->age << endl;

// 堆上的内存需要手动释放， 现在需要释放两次了
// 析构函数
p->~Animal();
// 释放空间
free(p);
```

#### 思考题
普通new出来的结构体或类，可以用`delete`来回收内存，那么`malloc`出来的空间，上面的例子走了一个析构，再走一个free，那么直接free会触发析构函数吗？

> 不会。